In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [4]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/codebert_embeddings/processed_embeddings/codebert_base_embed_line_by_line_avg.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.164304,0.293147,-0.025194,0.093005,0.101126,-0.240449,0.066143,0.028106,0.258849,-0.294083,...,0.643485,-0.093152,-0.008336,0.464331,-0.296749,-0.385003,0.395283,1,12150,critical
1,-0.122267,0.333413,-0.016818,0.069072,0.052436,-0.186081,0.013470,0.076470,0.219286,-0.257449,...,0.486227,-0.112189,0.010879,0.412307,-0.353872,-0.348544,0.363493,1,10168,critical
2,-0.204585,0.263288,-0.041261,0.118711,0.172630,-0.291919,0.095624,0.038220,0.274627,-0.312557,...,0.726426,-0.067420,-0.033730,0.540107,-0.246627,-0.438533,0.391139,1,12783,critical
3,-0.122016,0.238755,-0.054690,0.102714,0.183671,-0.253296,0.081947,0.029899,0.244510,-0.303301,...,0.730825,-0.105017,0.001282,0.462068,-0.248525,-0.410063,0.402719,1,12754,critical
4,-0.197520,0.270766,-0.057936,0.085830,0.159084,-0.282372,0.092354,0.032732,0.245176,-0.291814,...,0.674155,-0.068013,-0.025766,0.483451,-0.264792,-0.420454,0.388313,1,4277,critical


In [5]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [6]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8329
F1_macro: 0.5775
Accuracy: 0.8329
Precision_macro: 0.6015
Recall_macro: 0.5684

Bagging Metrics:
F1_micro: 0.8418
F1_macro: 0.5385
Accuracy: 0.8418
Precision_macro: 0.5824
Recall_macro: 0.5355

XGBoost Metrics:
F1_micro: 0.8333
F1_macro: 0.5792
Accuracy: 0.8333
Precision_macro: 0.6028
Recall_macro: 0.5702


In [7]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")



Final Test Results:

Random Forest Metrics:
F1_micro: 0.8185
F1_macro: 0.5613
Precision_macro: 0.5729
Recall_macro: 0.5556
Accuracy: 0.8185
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       786
           1       0.27      0.20      0.23       123

    accuracy                           0.82       909
   macro avg       0.57      0.56      0.56       909
weighted avg       0.80      0.82      0.81       909


Bagging Metrics:
F1_micro: 0.8449
F1_macro: 0.5542
Precision_macro: 0.6016
Recall_macro: 0.5468
Accuracy: 0.8449
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.96      0.91       786
           1       0.33      0.14      0.19       123

    accuracy                           0.84       909
   macro avg       0.60      0.55      0.55       909
weighted avg       0.80      0.84      0.82       909


XGBoost Metrics:
F1_micro: 0.8251
F1_macro: 

In [9]:
# Save results to a pickle file
output_path = "/Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/codebert/result_codebert_base_avg.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")



Results saved to /Users/mac/Desktop/Code_Smell_Detection/feature_envy/results/codebert/result_codebert_base_avg.pkl
